# STEP 2: Convert Text → Numbers (Tokenization & Vocabulary)
Neural Network can not understand the text it only understans the numbers

In [ ]:
import pandas as pd 
import numpy as np

train_df = pd.read_csv("training.csv")

## Define a Simple Tokenizer
Below function will do this : 
I am Tirth Patel  -- > ['i', 'am', 'tirth', 'patel']

In [ ]:
import re 
def tokenize(text):
    text = text.lower()
    text = re.sub(r"[^a-z\s]", "", text)
    return text.split()

print(tokenize("I am Tirth Patel")) 

## Build Vocabulary from Training Data

📌 Why special tokens? 
- `<PAD>` : for padding
- `<UNK>` : unseen words in test data

In [8]:
from collections import Counter

word_counter = Counter()

for sentance in train_df["text"]:
     tokens = tokenize(sentance)
     word_counter.update(tokens)
     
# Special tokens
PAD_TOKEN = "<PAD>"
UNK_TOKEN = "<UNK>"

vocab = {
    PAD_TOKEN: 0,
    UNK_TOKEN: 1
}

for word,_ in word_counter.items():
     vocab[word] = len(vocab)
     
print("Vocabulary size:",len(vocab))

Vocabulary size: 15214


## Convert Text to Numerical Sequences

In [10]:
def encode_sentence(sentance , vocab) :
     tokens = tokenize(sentance)
     return [vocab.get(token, vocab[UNK_TOKEN]) for token in tokens]

print(encode_sentence("I feel happy", vocab))

[2, 4, 661]


## Padding (VERY IMPORTANT)
Neural networks need same-length inputs.

In [12]:
MAX_LEN = 50

def pad_suquence(seq , max_len) :
     if len(seq) < max_len:
        return seq + [vocab[PAD_TOKEN]] * (max_len - len(seq))
     else:
          return seq[:max_len]

## Final Encoded Dataset (Training Only) 

In [16]:
X_train = [
     pad_suquence(encode_sentence(text , vocab) , MAX_LEN)
     for text in train_df['text']
]

y_train = train_df['label'].values
print(X_train[0])
print(len(X_train[0]))


[2, 3, 4, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
50


## What You’ve Achieved
Implemented custom tokenization, vocabulary construction, sequence encoding, and padding for NLP tasks.

# STEP 3: PyTorch Dataset & DataLoader (Industry Standard)
❗ Goal of this step
> Convert your processed data into a format that PyTorch models can train on.
---
## Why Dataset & DataLoader?
Instead of loading everything at once, PyTorch:
- Loads data in batches
- Shuffles training data
- Works efficiently on CPU / GPU

## Convert Data to PyTorch Tensors

In [ ]:
import torch 
from torch.utils.data import Dataset, DataLoader

## Create a Custom Dataset Class


In [ ]:
class EmotionDataset(Dataset) :
     def __init__(self , texts , labels) :
          self.texts = torch.tensor(texts, dtype=torch.long)
          self.labels = torch.tensor(labels ,dtype=torch.long )
     
     def __init__(self) :
          return len(self.labels)

     def __getitem__(self , idx) :
          return self.texts[idx], self.labels[idx]

## Create Training & Validation Datasets

In [ ]:
train_dataset = EmotionDataset(X_train, y_train)